In [2]:
import pandas as pd
import numpy as np
import xlearn

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.utils import shuffle

# Препроцессинг

In [17]:
data = pd.read_csv('../data.csv').sample(frac=0.2)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


## Анализ

In [18]:
data.drop(['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1',
                  'coeff_sum0', 'coeff_sum1'], axis=1, inplace=True)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1


In [19]:
data.drop('impressions', axis=1, inplace=True)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1


Столбец impressions состоит только из 1, можно он не несет никакой информации, удалил

In [20]:
data['date'] = data['date_time'].apply(lambda s: s.split()[0])

In [21]:
data['time'] = data['date_time'].apply(lambda s: s.split()[1])

In [22]:
data.drop('date_time', axis=1, inplace=True)
data.head()

,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,date,time
0,0,0,5664530014561852622,0,0,0,1,2021-09-27,00:01:30.000000
1,1,1,5186611064559013950,0,0,1,1,2021-09-26,22:54:49.000000
2,2,2,2215519569292448030,3,0,0,1,2021-09-26,23:57:20.000000
3,3,3,6262169206735077204,0,1,1,1,2021-09-27,00:04:30.000000
4,4,4,4778985830203613115,0,1,0,1,2021-09-27,00:06:21.000000


In [23]:
data.drop(data.index[[1390198]], inplace=True)

## feature_engineering

In [24]:
data['date'] = data['date'].apply(lambda s: int(s.split('-')[2]))
data['hour'] = data['time'].apply(lambda s: int(float(s.split(':')[0])))
data['minute'] = data['time'].apply(lambda s: int(float(s.split(':')[1])))
data['second'] = data['time'].apply(lambda s: int(float(s.split(':')[2])))
data.drop('time', axis=1, inplace=True)
data.head()

,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,date,hour,minute,second
0,0,0,5664530014561852622,0,0,0,1,27,0,1,30
1,1,1,5186611064559013950,0,0,1,1,26,22,54,49
2,2,2,2215519569292448030,3,0,0,1,26,23,57,20
3,3,3,6262169206735077204,0,1,1,1,27,0,4,30
4,4,4,4778985830203613115,0,1,0,1,27,0,6,21


In [25]:
data.date.unique()

array([27, 26, 29, 30, 28,  1,  2])

In [86]:
# l_hour = data.hour.unique().tolist()
# l_hour.sort()
# l_hour

In [87]:
# l_minute = data.minute.unique().tolist()
# l_minute.sort()
# l_minute

In [88]:
# l_second = data.second.unique().tolist()
# l_second.sort()
# l_second

In [26]:
data['date_sin'] = np.sin(data.date*(2.*np.pi/31))
data['date_cos'] = np.cos(data.date*(2.*np.pi/31))

data['hour_sin'] = np.sin(data.hour*(2.*np.pi/24))
data['hour_cos'] = np.cos(data.hour*(2.*np.pi/24))

data['minute_sin'] = np.sin((data.minute-1)*(2.*np.pi/60))
data['minute_cos'] = np.cos((data.minute-1)*(2.*np.pi/60))

data['second_sin'] = np.sin((data.second-1)*(2.*np.pi/60))
data['second_cos'] = np.cos((data.second-1)*(2.*np.pi/60))

In [27]:
# data.drop('date', axis=1, inplace=True)
data.drop('hour', axis=1, inplace=True)
data.drop('minute', axis=1, inplace=True)
data.drop('second', axis=1, inplace=True)
data.head()

,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,date,date_sin,date_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos
0,0,0,5664530014561852622,0,0,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.000000,1.000000,0.104528,-0.994522
1,1,1,5186611064559013950,0,0,1,1,26,-0.848644,0.528964,-0.500000,0.866025,-0.669131,0.743145,-0.951057,0.309017
2,2,2,2215519569292448030,3,0,0,1,26,-0.848644,0.528964,-0.258819,0.965926,-0.406737,0.913545,0.913545,-0.406737
3,3,3,6262169206735077204,0,1,1,1,27,-0.724793,0.688967,0.000000,1.000000,0.309017,0.951057,0.104528,-0.994522
4,4,4,4778985830203613115,0,1,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.500000,0.866025,0.866025,-0.500000


In [93]:
encoder = OneHotEncoder()
encoder.fit(data_categories)

OneHotEncoder()

In [94]:
data_categories_encodeed = encoder.transform(data_categories)

In [95]:
data_categories_encodeed.shape

(15821471, 5105)

In [96]:
data.drop(data_categories, axis=1, inplace=True)
data.head()

,campaign_clicks,clicks,date,date_sin,date_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos
0,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.000000,1.000000,0.104528,-0.994522
1,0,1,26,-0.848644,0.528964,-0.500000,0.866025,-0.669131,0.743145,-0.951057,0.309017
2,3,1,26,-0.848644,0.528964,-0.258819,0.965926,-0.406737,0.913545,0.913545,-0.406737
3,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.309017,0.951057,0.104528,-0.994522
4,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.500000,0.866025,0.866025,-0.500000


In [29]:
data.shape
data.head()

,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,date,date_sin,date_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos
0,0,0,5664530014561852622,0,0,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.000000,1.000000,0.104528,-0.994522
1,1,1,5186611064559013950,0,0,1,1,26,-0.848644,0.528964,-0.500000,0.866025,-0.669131,0.743145,-0.951057,0.309017
2,2,2,2215519569292448030,3,0,0,1,26,-0.848644,0.528964,-0.258819,0.965926,-0.406737,0.913545,0.913545,-0.406737
3,3,3,6262169206735077204,0,1,1,1,27,-0.724793,0.688967,0.000000,1.000000,0.309017,0.951057,0.104528,-0.994522
4,4,4,4778985830203613115,0,1,0,1,27,-0.724793,0.688967,0.000000,1.000000,0.500000,0.866025,0.866025,-0.500000


In [30]:
data.date.unique()

array([27, 26, 29, 30, 28,  1,  2])

In [31]:
test = data[data["date"] == 2] 
val = data[data["date"] == 1] 
train_dates = [27, 26, 28, 29, 30]
train = data[data["date"].isin(train_dates)]

In [32]:
y_test = data[data["date"] == 2].clicks
y_val = data[data["date"] == 1].clicks
y_train = data[data["date"].isin(train_dates)].clicks

In [33]:
train, y_train = shuffle(train, y_train)

In [36]:
train.head()

,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,date,date_sin,date_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos
15774271,12,156,2668952189763128257,0,1,6,0,26,-0.848644,0.528964,0.965926,0.258819,0.104528,-0.994522,0.743145,-0.669131
520048,27,46,2718200761269207542,1,4,0,0,30,-0.201299,0.979530,-0.707107,0.707107,-0.406737,-0.913545,0.406737,0.913545
8315801,30,39,7162333652790817158,0,0,7,0,29,-0.394356,0.918958,-0.965926,0.258819,-0.500000,0.866025,0.743145,0.669131
6951466,19,209,6586274245775978078,0,0,5,0,27,-0.724793,0.688967,-0.965926,0.258819,-0.309017,-0.951057,-0.978148,-0.207912
10697287,105,255,6954274397066355764,0,4,8,0,28,-0.571268,0.820763,-0.866025,0.500000,0.994522,-0.104528,0.500000,-0.866025


In [34]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
val.to_csv('train.csv', index=False)

In [35]:
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
y_val.to_csv('y_val.csv', index=False)

Код взят из этого гайда https://wngaw.github.io/field-aware-factorization-machines-with-xlearn/

In [49]:
def _convert_to_ffm(path, df, type, target, numerics, categories, features, encoder):
    # Flagging categorical and numerical fields
    print('convert_to_ffm - START')
    for x in numerics:
        if(x not in encoder['catdict']):
            encoder['catdict'][x] = 0
    for x in categories:
        if(x not in encoder['catdict']):
            encoder['catdict'][x] = 1

    nrows = df.shape[0]
    with open(path + str(type) + "_ffm.txt", "w") as text_file:

        # Looping over rows to convert each row to libffm format
        for n, r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow[target]))  # Set Target Variable here

            # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(encoder['catdict'].keys()):
                if(encoder['catdict'][x] == 0):
                    # Not adding numerical values that are nan
                    if math.isnan(datarow[x]) is not True:
                        datastring = datastring + " "+str(i)+":" + str(i)+":" + str(datarow[x])
                else:

                    # For a new field appearing in a training example
                    if(x not in encoder['catcodes']):
                        encoder['catcodes'][x] = {}
                        encoder['currentcode'] += 1
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    # For already encoded fields
                    elif(datarow[x] not in encoder['catcodes'][x]):
                        encoder['currentcode'] += 1
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    code = encoder['catcodes'][x][datarow[x]]
                    datastring = datastring + " "+str(i)+":" + str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

    # print('Encoder Summary:')
    # print(json.dumps(encoder, indent=4))
    return encoder

In [50]:
data_columns = list(train.columns)
data_columns.remove('clicks')

In [51]:
encoder = {"currentcode": 0,
           "catdict": {},
           "catcodes": {}}

In [53]:
encoder = _convert_to_ffm('data/', train, 'train', 'clicks', [], data_columns, [], encoder)

convert_to_ffm - START


In [54]:
encoder = _convert_to_ffm('data/', val, 'val', 'clicks', [], data_columns, [], encoder)

convert_to_ffm - START


In [55]:
encoder = _convert_to_ffm('data/', test, 'test', 'clicks', [], data_columns, [], encoder)

convert_to_ffm - START


# CrossValidation на полном датасете

In [1]:
import xlearn as xl

In [2]:
for k in [2, 4]:
    for l in [0.000001]:
        param = {
            'task': 'binary',
             'lr': 0.1,
             'lambda': l,
             'metric': 'auc', 
             'k': k,
             'epoch': 10
        }
        print(f'k = {k}, l = {l}')
        ffm_model = xl.create_ffm()
        ffm_model.setTrain("data/train_ffm.txt")
        ffm_model.cv(param)

k = 2, l = 1e-06
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[ WARNING    ] Cross-validation doesn't support early-stopping. xLearn has already close early-stopping.
[------------] xLearn uses 4 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data/train_ffm.txt_0.bin) NOT found. Convert text file to binary file.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data/train_ffm.txt_1.bin) NOT found. Convert text file to binary file.
[

# Train на полном датасете (так и не дождался)

In [1]:
import xlearn as xl

In [2]:
param = {
    'task': 'binary',
     'lr': 0.1,
     'lambda': 0.000001,
     'metric': 'auc', 
     'k': 4,
     'epoch': 10,
     'nthread': 10
}

In [ ]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("data/train_ffm.txt")
ffm_model.setValidate("data/val_ffm.txt")
ffm_model.setTest("data/test_ffm.txt")
ffm_model.setSigmoid()
ffm_model.fit(param, 'model.out')

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 10 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data/train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data/val_ffm.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 5666483
[------------] Number of Field: 15
[------------] Time cost for reading problem: 27.13 (sec)
[ ACT

# Train на датасете размером 0.2 от исходного 

In [2]:
import xlearn as xl

In [3]:
param = {
    'task': 'binary',
     'lr': 0.1,
     'lambda': 0.000001,
     'metric': 'auc', 
     'k': 4,
     'epoch': 10,
     'nthread': 6
}

In [4]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("data02/train_ffm.txt")
ffm_model.setValidate("data02/val_ffm.txt")
ffm_model.setTest("data02/test_ffm.txt")
ffm_model.setSigmoid()
ffm_model.fit(param, 'model.out')

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 6 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data02/train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data02/val_ffm.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 1798313
[------------] Number of Field: 15
[------------] Time cost for reading problem: 3.81 (sec)
[ A

In [5]:
ffm_model.predict("model.out", "output/predictions01.txt")

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 4 threads for prediction task.
[ ACTION     ] Load model ...
[------------] Load model from model.out
[------------] Loss function: cross-entropy
[------------] Score function: ffm
[------------] Number of Feature: 1798313
[------------] Number of K: 4
[------------] Number of field: 15
[------------] Time cost for loading model: 0.74 (sec)
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data02/test_ffm.txt.bin) NOT found. Convert text file to binary file

In [12]:
from sklearn.metrics import roc_auc_score, log_loss
import pandas as pd

In [16]:
y_test = pd.read_csv('y_true.csv')
y_test.head()

,clicks
0,0
1,0
2,0
3,0
4,0


In [17]:
y_pred = pd.read_csv('output/predictions01.txt', header=None)
my_log_loss = log_loss(y_test, y_pred)
my_auc = roc_auc_score(y_test, y_pred)
print(f'======= Test Metrics =======')
print(f'======= FFM Model =======')
print(f'log_loss: {my_log_loss}')
print(f'auc: {my_auc}')

======= Test Metrics =======
======= FFM Model =======
log_loss: 0.14684748004840417
auc: 0.7525666828260666


In [18]:
param = {
    'task': 'binary',
     'lr': 0.1,
     'lambda': 0.000001,
     'metric': 'auc', 
     'k': 4,
     'epoch': 50,
     'nthread': 8
}

In [19]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("data02/train_ffm.txt")
ffm_model.setValidate("data02/val_ffm.txt")
ffm_model.setTest("data02/test_ffm.txt")
ffm_model.setSigmoid()
ffm_model.fit(param, 'model1.out')

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 8 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data02/train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data02/val_ffm.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 1798313
[------------] Number of Field: 15
[------------] Time cost for reading problem: 5.07 (sec)
[ A

In [20]:
ffm_model.predict("model1.out", "output/predictions02.txt")

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 4 threads for prediction task.
[ ACTION     ] Load model ...
[------------] Load model from model1.out
[------------] Loss function: cross-entropy
[------------] Score function: ffm
[------------] Number of Feature: 1798313
[------------] Number of K: 4
[------------] Number of field: 15
[------------] Time cost for loading model: 0.61 (sec)
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (data02/test_ffm.txt.bin) found. Skip converting text to binary.
[--

In [21]:
y_pred = pd.read_csv('output/predictions02.txt', header=None)
my_log_loss = log_loss(y_test, y_pred)
my_auc = roc_auc_score(y_test, y_pred)
print(f'======= Test Metrics =======')
print(f'======= FFM Model =======')
print(f'log_loss: {my_log_loss}')
print(f'auc: {my_auc}')

======= Test Metrics =======
======= FFM Model =======
log_loss: 0.14598398800929485
auc: 0.7543123284241735


## Train
Получили метрики хуже, чем у линейной модели
Это в первую очередь связанно с тем что мы обучались на 20% исходного датасета. 
Ниже приведенны старые метрики

In [107]:
y_pred = model.predict_proba(dataset_test)[:, 1]
my_log_loss = log_loss(y_test, y_pred)
my_auc = roc_auc_score(y_test, y_pred)
print(f'======= Test Metrics =======')
print(f'======= My Model =======')
print(f'log_loss: {my_log_loss}')
print(f'auc: {my_auc}')

======= Test Metrics =======
======= My Model =======
log_loss: 0.1334549924173812
auc: 0.7801157566633397


In [108]:
y_pred_baseline = np.full(y_test.shape, np.mean(y_train))
baseline_log_loss = log_loss(y_test, y_pred_baseline)
baseline_auc = roc_auc_score(y_test, y_pred_baseline)
print(f'======= Baseline =======')
print(f'log_loss: {baseline_log_loss}')
print(f'auc: {baseline_auc}')

======= Baseline =======
log_loss: 0.15486197934508547
auc: 0.5


Наша модель побила baseline